In [ ]:
!pip install -qU accelerate transformers bitsandbytes peft datasets sentencepiece tqdm huggingface evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    TaskType
)
from huggingface_hub import login
import gdown

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/NLP/Datasets/t5_lyrics_chords_clean_v2.csv")
print(f"Dataset loaded with {len(df)} examples")

dataset = Dataset.from_pandas(df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset loaded with 50000 examples


In [ ]:
from huggingface_hub import login
login()

In [ ]:
model_name = "TinyLlama/TinyLlama_v1.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [ ]:
base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


In [ ]:
def format_prompt(example):
    return f"""### Input Lyrics:
{example['input_text']}

### Chord Progression:
{example['target_text']}</s>"""

def tokenize(example):
    prompt = f"""### Input Lyrics:
{example['input_text']}

### Chord Progression:
{example['target_text']}</s>"""

    encoded = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    input_only = f"### Input Lyrics:\n{example['input_text']}\n\n### Chord Progression:\n"
    input_len = len(tokenizer(input_only, add_special_tokens=False)["input_ids"])

    labels = encoded["input_ids"][:]
    for i in range(input_len):
        if i < len(labels):
            labels[i] = -100

    labels = labels[:512]
    if len(labels) < 512:
        labels += [-100] * (512 - len(labels))

    return {
        "input_ids": encoded["input_ids"][:512],
        "attention_mask": encoded["attention_mask"][:512],
        "labels": labels
    }


In [ ]:
tokenized_dataset = dataset.map(
    tokenize,
    batched=False,
    remove_columns=dataset.column_names
)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict

# Split into train and eval
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


In [ ]:
output_dir = "/content/drive/MyDrive/tinyllama_chord_gen"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created output directory: {output_dir}")
else:
    print(f"Output directory already exists: {output_dir}")

from transformers import TrainingArguments

output_dir = "/content/drive/MyDrive/tinyllama_chord_gen" if os.path.exists('/content/drive') else "tinyllama_chord_gen"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=3e-4,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=10,
    load_best_model_at_end=True,
    fp16=True,
    report_to="tensorboard",
    optim="adamw_torch",
    warmup_ratio=0.03,
    weight_decay=0.01,
    max_grad_norm=0.5,
    remove_unused_columns=False,
    gradient_checkpointing=True,
)



Output directory already exists: /content/drive/MyDrive/tinyllama_chord_gen


In [ ]:
from transformers import DataCollatorForLanguageModeling, Trainer

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
print("Starting training...")
trainer.train()


Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
100,2.849400,2.789057
200,2.831100,2.788833
300,2.591100,2.788769
400,2.719700,2.788880
500,2.766900,2.788850
600,2.598400,2.788618
700,2.634400,2.791786
800,2.780100,2.791598
900,2.733800,2.789905
1000,2.916100,2.789664


TrainOutput(global_step=16875, training_loss=5.490655468297888, metrics={'train_runtime': 32026.1363, 'train_samples_per_second': 4.215, 'train_steps_per_second': 0.527, 'total_flos': 4.3090153832448e+17, 'train_loss': 5.490655468297888, 'epoch': 3.0})

In [ ]:
save_path = f"{output_dir}/final_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"\nModel saved to {save_path}")


Model saved to /content/drive/MyDrive/tinyllama_chord_gen/final_model
